In [ ]:
import ROOT
ROOT.gStyle.SetOptStat(0) # deactivate annoying stat box in histograms

In [ ]:
# Load a root file to a chain
TChiWH_750_1 = ROOT.TChain("Events")
TChiWH_750_1.Add("WH_had_750_1_v1.root")

In [ ]:
# Explore the content of the file
TChiWH_750_1.ls() # shows you the ROOT trees that you loaded
TChiWH_750_1.GetListOfBranches().ls() # shows the branches of the tree

#TChiWH_750_1.Scan("MET_pt:MET_phi")

In [ ]:
## Histogramming
# That's what we usually mean when we say "Let's make a plot".

can = ROOT.TCanvas("can", "", 700,700)
myFirstHisto = ROOT.TH1F("myFirstHisto", "", 10, -0.5, 9.5)
nJet = "Sum$(Jet_pt>30&&abs(Jet_eta)<2.4&&Jet_jetId>1)"
TChiWH_750_1.Draw("%s>>myFirstHisto"%nJet)

can.Draw()

In [ ]:
## Formating a plot
# Let's make the plot a bit more beautiful, and normalize it to 1.

myFirstHisto.Scale(1/myFirstHisto.Integral())
myFirstHisto.GetXaxis().SetTitle("N_{jet}")
myFirstHisto.GetYaxis().SetTitle("frac. of events")
myFirstHisto.GetYaxis().SetTitleOffset(1.5)
myFirstHisto.SetLineColor(ROOT.kRed+2)
myFirstHisto.SetLineWidth(2)
myFirstHisto.Draw("hist")
can.Draw()

In [ ]:
from helpers import * # import some functions.

cut = "Sum$(Jet_pt>30&&abs(Jet_eta)<2.4&&Jet_jetId>1)==4"

mbb_histo = ROOT.TH1F("mbb_histo", "", 50,20,220)
mqq_histo = ROOT.TH1F("mqq_histo", "", 50,20,220)

TChiWH_750_1.Draw('>>eList',cut)
elist = ROOT.gDirectory.Get("eList")
number_events = elist.GetN()
for i in range(number_events):
    
    # We expect two b-jets
    b1 = ROOT.TLorentzVector()
    b2 = ROOT.TLorentzVector()

    # and two light jets
    q1 = ROOT.TLorentzVector()
    q2 = ROOT.TLorentzVector()
    
    update_progress(float(i)/number_events)
    TChiWH_750_1.GetEntry(elist.GetEntry(i))
    
    jets     = getJets(TChiWH_750_1)
    bJets    = getBJets(TChiWH_750_1, year=2018)
    nonBJets = getNonBJets(TChiWH_750_1, year=2018)
    
    # only use the case where the event is well defined: 2 b-tagged jets, 2 non-b-tagged jets
    if len(bJets)==2:

        #print len(nonBJets)
        
        b1.SetPtEtaPhiM(bJets[0]['pt'], bJets[0]['eta'], bJets[0]['phi'], 0)
        b2.SetPtEtaPhiM(bJets[1]['pt'], bJets[1]['eta'], bJets[1]['phi'], 0)
        q1.SetPtEtaPhiM(nonBJets[0]['pt'], nonBJets[0]['eta'], nonBJets[0]['phi'], 0)
        q2.SetPtEtaPhiM(nonBJets[1]['pt'], nonBJets[1]['eta'], nonBJets[1]['phi'], 0)
        
        mbb = (b1+b2).M()
        mbb_histo.Fill(mbb)
        
        mqq = (q1+q2).M()
        mqq_histo.Fill(mqq)

In [ ]:
can2 = ROOT.TCanvas("can2", "", 700,700)

mbb_histo.SetLineColor(ROOT.kGreen+2)
mbb_histo.SetLineWidth(2)
mbb_histo.GetXaxis().SetTitle("di-jet mass (GeV)")

mqq_histo.SetLineColor(ROOT.kBlue+1)
mqq_histo.SetLineWidth(2)

mbb_histo.Draw("hist")
mqq_histo.Draw("hist same")
can2.Draw()

In [ ]:
FatJetMass_histo = ROOT.TH1F("FatJetMass_histo", "", 50,20,220)
cut = "Sum$(Jet_pt>30&&abs(Jet_eta)<2.4&&Jet_jetId>1)==4&&Sum$(Jet_pt>30&&abs(Jet_eta)<2.4&&Jet_jetId>1&&Jet_btagDeepB>0.4184)==2"
TChiWH_750_1.Draw("FatJet_msoftdrop>>FatJetMass_histo")

can3 = ROOT.TCanvas("can3", "", 700,700)
FatJetMass_histo.Draw("hist")
can3.Draw()

In [ ]:
FatJetMass_Higgs_histo = ROOT.TH1F("FatJetMass_Higgs_histo", "", 50,20,220)
FatJetMass_W_histo = ROOT.TH1F("FatJetMass_W_histo", "", 50,20,220)

TChiWH_750_1.Draw("FatJet_mass>>FatJetMass_Higgs_histo", "FatJet_deepTag_H>0.6")
TChiWH_750_1.Draw("FatJet_mass>>FatJetMass_W_histo", "FatJet_deepTag_WvsQCD>0.8")

FatJetMass_W_histo.SetLineColor(ROOT.kGreen+2)
FatJetMass_W_histo.SetLineWidth(2)
FatJetMass_W_histo.GetXaxis().SetTitle("FatJet mass (GeV)")

FatJetMass_Higgs_histo.SetLineColor(ROOT.kBlue+1)
FatJetMass_Higgs_histo.SetLineWidth(2)

can4 = ROOT.TCanvas("can4", "", 700,700)
FatJetMass_W_histo.Draw("hist")
FatJetMass_Higgs_histo.Draw("hist same")

can4.Draw()

In [ ]:
## Signal vs Noise

TChiWH_500_150 = ROOT.TChain("Events")
TChiWH_500_150.Add("WH_had_500_150_v1.root")

TTJets = ROOT.TChain("Events")
TTJets.Add("TTJets.root")

In [ ]:
#cut = "Sum$(Jet_pt>30&&abs(Jet_eta)<2.4&&Jet_jetId>1)==4&&Sum$(Jet_pt>30&&abs(Jet_eta)<2.4&&Jet_jetId>1&&Jet_btagDeepB>0.4184)==2"
cut = "(1)"

Signal1_MET_histo = ROOT.TH1F("Signal1_MET_histo", "", 50,0,500)
Signal2_MET_histo = ROOT.TH1F("Signal2_MET_histo", "", 50,0,500)
TTJets_MET_histo  = ROOT.TH1F("TTJets_MET_histo",  "", 50,0,500)

TChiWH_750_1   .Draw("MET_pt>>Signal1_MET_histo", cut)
TChiWH_500_150 .Draw("MET_pt>>Signal2_MET_histo", cut)
TTJets         .Draw("MET_pt>>TTJets_MET_histo",  cut)


In [ ]:
Signal1_MET_histo .SetLineColor(ROOT.kBlue+1)
Signal2_MET_histo .SetLineColor(ROOT.kGreen+2)
TTJets_MET_histo  .SetLineColor(ROOT.kRed+1)

leg = ROOT.TLegend(0.6,0.4,0.85,0.5)


can5 = ROOT.TCanvas("can5", "", 700,700)

same = ''
for h in [TTJets_MET_histo, Signal1_MET_histo, Signal2_MET_histo]:
    h.SetLineWidth(2)
    h.Scale(1/h.Integral())
    
    h.GetXaxis().SetTitle("p_{T}^{miss} (GeV)")
    
    h.Draw('hist'+same)
    same = ' same'

leg.AddEntry(Signal1_MET_histo, 'TChiWH(750,1)', 'l')
leg.AddEntry(Signal2_MET_histo, 'TChiWH(500,150)', 'l')
leg.AddEntry(TTJets_MET_histo, 't#bar{t}+jets', 'l')

leg.Draw()
can5.SetLogy()
can5.Draw()